In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.40)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.72 sec - Loss 0.090179 - ACC 70.30% - ACC Mean 70.30% - AUC 79.44% - AUC Mean 79.44% - Deter 000
Ite 00050 - 0.11 sec - Loss 0.085490 - ACC 72.10% - ACC Mean 70.94% - AUC 79.21% - AUC Mean 79.34% - Deter 038
Ite 00100 - 0.12 sec - Loss 0.074606 - ACC 71.84% - ACC Mean 71.23% - AUC 80.28% - AUC Mean 78.90% - Deter 001
Ite 00150 - 0.15 sec - Loss 0.063214 - ACC 71.28% - ACC Mean 71.41% - AUC 79.93% - AUC Mean 79.81% - Deter 051
Ite 00200 - 0.13 sec - Loss 0.059992 - ACC 71.28% - ACC Mean 71.39% - AUC 79.96% - AUC Mean 79.96% - Deter 101
Ite 00250 - 0.12 sec - Loss 0.059210 - ACC 71.80% - ACC Mean 71.43% - AUC 80.27% - AUC Mean 79.93% - Deter 151
Ite 00300 - 0.12 sec - Loss 0.058822 - ACC 71.78% - ACC Mean 71.40% - AUC 79.59% - AUC Mean 79.92% - Deter 201
Ite 00350 - 0.15 sec - Loss 0.058482 - ACC 71.28% - ACC Mean 71.32% - AUC 80.06% - AUC Mean 79.86% - Deter 251
Ite 00400 - 0.10 sec - Loss 0.058135 - ACC 71.03% - ACC Mean 71.02% - AUC 79.54% - AUC Mean 79.53% - Deter 301
I

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 72.24% +- 0.0% - auc: 78.79% +- 0.0%
MEAN - acc: 71.73% +- 0.0% - auc: 78.94% +- 0.0%
KNN - acc: 74.49% +- 0.0% - auc: 81.73% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:39<00:00, 126.62it/s]



GAIN RUN 1/10 - acc: 75.84% +- 7.54% - auc: 79.68% +- 5.67%



100%|██████████| 5000/5000 [00:32<00:00, 152.82it/s]



GAIN RUN 2/10 - acc: 76.31% +- 6.61% - auc: 82.21% +- 2.47%



100%|██████████| 5000/5000 [00:37<00:00, 132.81it/s]


...


GAIN RUN 8/10 - acc: 77.13% +- 7.17% - auc: 81.57% +- 4.65%



100%|██████████| 5000/5000 [00:36<00:00, 135.44it/s]



GAIN RUN 9/10 - acc: 78.41% +- 4.42% - auc: 82.07% +- 4.04%



100%|██████████| 5000/5000 [00:40<00:00, 122.39it/s]



GAIN RUN 10/10 - acc: 72.7% +- 6.43% - auc: 76.47% +- 7.5%

GAIN GLOBAL - acc: 75.36% +- 2.03% - auc: 80.19% +- 1.82%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 70.99% +- 0.0% - auc: 78.35% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 1/10 - acc: 72.47% +- 2.91% - auc: 77.57% +- 4.18%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 2/10 - acc: 71.5% +- 2.97% - auc: 78.14% +- 3.47%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 3/10 - acc: 71.55% +- 5.35% - auc: 80.09% +- 5.42%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 4/10 - acc: 70.86% +- 5.23% - auc: 78.03% +- 4.86%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 5/10 - acc: 72.36% +- 3.4% - auc: 79.91% +- 4.44%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8

MISSFOREST RUN 6/10 - acc: 70.9% +- 5.76% - auc: 78.97% +- 6.03%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration:

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 76.23% +- 3.48% - auc: 82.13% +- 2.44%
SOFTIMPUTE RUN 2/10 - acc: 72.55% +- 4.91% - auc: 79.93% +- 5.37%
SOFTIMPUTE RUN 3/10 - acc: 73.93% +- 3.59% - auc: 80.24% +- 4.18%
SOFTIMPUTE RUN 4/10 - acc: 72.58% +- 4.82% - auc: 80.66% +- 5.62%
SOFTIMPUTE RUN 5/10 - acc: 75.13% +- 2.88% - auc: 81.26% +- 2.69%
SOFTIMPUTE RUN 6/10 - acc: 73.87% +- 1.1% - auc: 82.29% +- 2.07%
SOFTIMPUTE RUN 7/10 - acc: 74.79% +- 3.51% - auc: 79.54% +- 3.68%
SOFTIMPUTE RUN 8/10 - acc: 70.41% +- 3.99% - auc: 77.17% +- 3.26%
SOFTIMPUTE RUN 9/10 - acc: 76.57% +- 4.63% - auc: 80.75% +- 2.42%
SOFTIMPUTE RUN 10/10 - acc: 77.43% +- 2.27% - auc: 83.44% +- 1.79%

SOFTIMPUTE GLOBAL - acc: 74.35% +- 2.02% - auc: 80.74% +- 1.64%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4114

SINKHORN RUN 1/10 - acc: 73.34% +- 3.64% - auc: 79.52% +- 2.76%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.2672

SINKHORN RUN 2/10 - acc: 73.28% +- 3.58% - auc: 80.0% +- 4.53%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.2777

SINKHORN RUN 3/10 - acc: 73.92% +- 4.65% - auc: 82.04% +- 2.66%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.1077

SINKHORN RUN 4/10 - acc: 76.25% +- 1.36% - auc: 81.73% +- 3.12%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4741

SINKHORN RUN 5/10 - acc: 74.67% +- 4.12% - auc: 79.94% +- 3.74%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4108

SINKHORN RUN 6/10 - acc: 73.65% +- 2.74% - auc: 81.38% +- 3.89%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.1520

SINKHORN RUN 7/10 - acc: 73.08% +- 3.45% - auc: 79.65% +- 1.47%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.2293

SINKHORN RUN 8/10 - acc: 75.24% +- 0.95% - auc: 81

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 70.32% +- 3.49% - auc: 80.23% +- 2.84%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 72.08% +- 6.1% - auc: 76.96% +- 3.89%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 74.02% +- 3.64% - auc: 81.8% +- 1.67%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 70.27% +- 4.01% - auc: 79.56% +- 4.1%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 71.59% +- 3.2% - auc: 79.3% +- 3.46%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 72.99% +- 3.82% - auc: 79.84% +- 3.08%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 71.18% +- 4.73% - auc: 79.55% +- 3.31%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 71.29% +- 3.02% - auc: 78.99% +- 3.37%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 7

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is even with KNN
Metric AUC - OUR METHOD is even with KNN
Metric ACC - OUR METHOD is even with GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is even with SOFT
Metric AUC - OUR METHOD is even with SOFT
Metric ACC - OUR METHOD is even with SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly better than MIDA
